In [1]:
import numpy as np
import itertools as it
import matplotlib.pyplot as plt


### Part 1: ###
Convert uberIncomesRaw and carCost in the following cell to valuations per minute, or per mile (depending on usage of Google Distance Matrix API)

In [3]:
# Median Uber user has an income of approximately $71,000
# 40% of Uber passengers make at least $100,000
# from http://uctc.net/research/papers/UCTC-FR-2014-08.pdf
# I added some high earning users, and the result is an
# approximate user distribution of incomes in 2014:

uberIncomesRaw = [25.0, 50.0, 85.0, 150.0, 300.0, 600.0]
uberIncomesDistRaw = [8.0,23.0,18.0,27.0,9.0, 2.0] 
# Note that this doesn't sum to 100, because about 20% of uber drivers
# declined to share their income with the surveyors

uberIncomes = uberIncomesRaw/(np.ones(len(uberIncomesRaw)))
# Note that these numbers are not normalized in any way
# In production, these numbers should be in units of $/minute

uberIncomesDist = uberIncomesDistRaw/np.sum(uberIncomesDistRaw)
# Force probabilities to sum to 1.

#Estimated car value of time to be equivalent to 15K/year earner
carCost = 15

### Part 2: ###

Request user input of destinations, including an origin point and a number of dropoff points. Then, convert these destinations to two-dimensional coordinates and return them as: $$[(originx,originy,0.0),(x_1,y_1,1.0),(x_2,y_2,2.0),...]$$

At this stage, it would be OK if we just get something fairly reasonable that we can feed to Google Maps and use to check the rest of the results; but in theory we should be able to type in or somehow input d

In [7]:
def destinations(inputs):
    return 0

### Part 3: ###
Use the destinations to make a Google Distance Matrix API using all of our destinations (notice that you'll have to strip out the indexing of the locations) as both origin and destination points (so if there are $n$ destinations, the output should be an $n+1 \times n+1$ matrix). Also it would be great if the returned matrix could just be floats representing the number of minutes between locations; see if parsing is easy. Make sure that the order of things is preserved.

In [8]:
def distanceMatrix(destinations):
    return 0

The below code is slightly modified from the version in the abstract locatiosn to reflect that our list of destinations will look like $$[origin, loc_1, loc_2, \ldots]$$

In [11]:
def pathList(destinations):
    perms = it.permutations(destinations[1:len(destinations)])
    origin = destinations[0]
    lst = list(perms)
    np.random.shuffle(lst)

    #To reduce computation time in the case of a large number of riders,
    #We simply look at some large subset of paths instead of the entire space
    #Obviously this isn't great in the worst case, but it is good in the average case
    if len(lst) > 120:
        lst = lst[:120]
    permlist = []
    for x in lst:
        newpath = []
        newpath.append(origin)
        for j in x:
            newpath.append(j)
        permlist.append(newpath)

    return permlist

def subsetPathList(destinations, leftOut):
    subDests = []
    for x in destinations:
        if x[2] != leftOut:
            subDests.append(x)
            
    return pathList(subDests)

This code is copied from the abstract location version; nothing has changed.

In [13]:
# Compute the cost of a given path given a distance matrix
# containing pairwise distances between points in the path
def costMat(weights, path, distances):
    currloc = path[0]
    currcost = 0
    for x in path:
        currcost += weights[x[2]]   
    totalcost = 0
    for i in range(len(path)):
        totalcost += currcost * distances[currloc[2]][path[i][2]]
        if i > 0:
            currcost = currcost - weights[path[i][2]]
        currloc = path[i]
    return totalcost

# Compute the environmental cost of the path, equal to
# the cost of the path to the car
def envCostMat(path, distances):
    currloc = path[0]
    currcost = carCost
    totalcost = 0
    for i in range(len(path)):
        if i > 0:
            totalcost += currcost * distances[currloc[2]][path[i][2]]
            currloc = path[i]
    return totalcost

# Computes the individual costs of a given path and returns a matrix
# with the respective costs formatted as [car, 0, 1, . . . ]
def indivCostMat(weights, path, distances):
    numPeople = riders+1
    costs = np.zeros(numPeople)
    currloc = path[0]
    totalcost = 0
    inCar = set()
    inCar.add(0)
    for n in range(len(path)):
        inCar.add(path[n][2])
    for i in range(len(path)):
        for j in inCar:
            costs[j] += weights[j]*distances[currloc[2]][path[i][2]]
            
        if i > 0: #Never removes the car
            inCar.remove(path[i][2])
        currloc = path[i]
    return costs

def optimalPath(weights, paths, distances):
    wgtcost =[]
    for i in range(len(paths)):
        wgtcost.append(costMat(weights,paths[i],distances))
    
    optimal = wgtcost.index(min(wgtcost))
    return optimal

def shortestPath(paths,distances):
    unwgtcost = []
    for i in range(len(paths)):
        unwgtcost.append(envCostMat(paths[i],distances))
    
    shortest =  unwgtcost.index(min(unwgtcost))
    return shortest




This code is unique to calculating payments, and is unfinished. I think it works as desired, although you may want to compare and experiment with the less polished code in the GlenProject-16-11-13 file to make sure that this does what you want.

In [15]:
def harmMatrices(timeValues, destinations):
    distMatrix = distanceMatrix(destinations)
    allPaths = pathList(destinations)
    optPath = allPaths[optimalPath(timeValues,allPaths,distMatrix)]
    deficientPaths = []
    for i in range(len(timeValues)):
        deficientPaths.append(subsetPathList(dests,i))
    subOptCosts = []
    for i in range(len(timeValues)):
        deficientOptPath = deficientPaths[i][optimalPath(vals,deficientPaths[i],distMatrix)]
        subOptCosts.append(indivCostMat(vals, deficientOptPath,distMatrix))
    output = []
    output[0] = subOptCosts[0]
    for i in range(len(timeValues)):
        if i > 0:
            payments = subOptCosts[0]-subOptCosts[i]
            payments[i] = 0
            output[i] = payments
        
    return output

   

        
        


In [16]:
for i in range(len(output)): 
    if i == 0:
        print "Total cost to passengers is ", output[i]
    else:
        print "Marginal impact of rider ", i, "= ", output[i]

NameError: name 'output' is not defined

### Part 4: ###

Basically what remains to be done is to turn this output (marginal impact of riders) into a payment scheme. To that end, please read a paper about Vickrey Clarke Groves mechanisms and make sure that you know and can defend the answers to the following questions: 

1. What is the right way to compute payments in a VCG auctions?
2. What does the mechanism do with thsoe payment?

Then, find a way to use the harmMatrices (which will have units of dollars after the Distance Matrix API stuff is implemented) to compute appropriate payments from each participant. Try to see if these payments are reasonable, and if they are not, to figure out what we are doing wrong.
